## An Agentic RAG System
An agentic RAG system on Computer Science domain knowledge


#### Importing require libraries

In [1]:
import os,time
from dotenv import load_dotenv
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from typing import Literal
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
import requests
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, TimeoutError as FutureTimeout


#### Load api key and intialize llm

In [2]:
load_dotenv()
llm= ChatOpenAI(model='gpt-4o-mini', temperature=0)

### Load PDF with metadata Document

In [3]:
def _load_pdf(file_path:str):
    loader= PyPDFLoader(file_path)
    return loader.load()

In [ ]:
def load_pdf_with_metadata(file_path:str,document_name:str, institution:str="University of Ibadan", timeout_sec=60):
    with ProcessPoolExecutor(max_workers=1) as ex:
        file= ex.submit(_load_pdf,str(file_path))
        try:
            pages= file.result(timeout=timeout_sec)
        except FutureTimeout:
            raise TimeoutError(f"Timed out after {timeout_sec}s while parsing {file_path.name}")
        
        
        for page in pages:
            page.metadata.update({
                "document_name": document_name,
                "institution": institution,
                "page_no": (page.metadata.get("page") or 0)+1,
                "source_file": str(file_path),
            })
        return pages

# Now using the function to load the pdf files
documents=[]
bad_files=[]

doc_dir= Path(os.getenv("DOCS_DIR"))
pdfs= sorted(doc_dir.glob("*.pdf"))

if not pdfs:
    print ("No pdf file found")

for i, pdf in enumerate(pdfs,1):
    print(f"[{i}/{len(pdfs)}] Loading: {pdf.name}....", flush=True)
    to= time.time()
    try:
        pages=load_pdf_with_metadata(pdf,pdf.name, timeout_sec=60)
        documents+=pages
        print(f"OK: {len(pages)} pages in {time.time()-to:.2f}s", flush=True)
    except Exception as e:
        bad_files.append((pdf.name, str(e)))
        print(f"    SKIP: {pdf.name} -> {e}", flush=True)
# Review loaded docs
print(f"Total loaded pages: {len(documents)}")
if bad_files:
    print("\nSkipped files:")
    for name, err in bad_files:
        print(f"file name:{name}, Error: {err}")

for doc in documents[:3]:
    print(doc.metadata)



[1/5] Loading: FINAL_Student_Hand_Book.pdf....


    SKIP: FINAL_Student_Hand_Book.pdf -> A process in the process pool was terminated abruptly while the future was running or pending.
[2/5] Loading: GENDER_POLICY_DOMMY.pdf....
    SKIP: GENDER_POLICY_DOMMY.pdf -> A process in the process pool was terminated abruptly while the future was running or pending.
[3/5] Loading: handbook.pdf....
    SKIP: handbook.pdf -> A process in the process pool was terminated abruptly while the future was running or pending.
[4/5] Loading: MANUAL_OF_STYLE.pdf....
    SKIP: MANUAL_OF_STYLE.pdf -> A process in the process pool was terminated abruptly while the future was running or pending.
[5/5] Loading: SEXUAL_HASSASSMENT_HANDOUT_DOMMY.pdf....
    SKIP: SEXUAL_HASSASSMENT_HANDOUT_DOMMY.pdf -> A process in the process pool was terminated abruptly while the future was running or pending.
Total loaded chunks: 0

Skipped files:
file name:FINAL_Student_Hand_Book.pdf, Error: A process in the process pool was terminated abruptly while the future was running 

In [8]:
from pathlib import Path
import os

doc_dir = Path(os.getenv("DOCS_DIR"))
pdfs = sorted(doc_dir.glob("*.pdf"))

for p in pdfs:
    with open(p, "rb") as f:
        head = f.read(8)
    print(p.name, "->", head, "| size:", p.stat().st_size)


FINAL_Student_Hand_Book.pdf -> b'%PDF-1.7' | size: 317143865
GENDER_POLICY_DOMMY.pdf -> b'%PDF-1.3' | size: 1010161
handbook.pdf -> b'%PDF-1.5' | size: 30534951
MANUAL_OF_STYLE.pdf -> b'%PDF-1.6' | size: 201314
SEXUAL_HASSASSMENT_HANDOUT_DOMMY.pdf -> b'%PDF-1.3' | size: 900965


### Chunking the documents

In [7]:
## Chunk document
def chunking (document):
    

_IncompleteInputError: incomplete input (4145246553.py, line 3)